In [2]:
import pandas as pd
import numpy
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.metrics import mean_squared_error

In [3]:
# loading train dataframe
dir = 'C:/Users/sgulbin/Work/Personal/house_prices_regr/'
df = pd.read_csv(dir+'Kaggle/train.csv')

# loading better column names
cols_dic = pd.read_csv(dir+'cols_dic.csv')

# appending column names to the dataframe
df.columns = cols_dic.description.tolist()

# dropping useless columns
df = df.drop('Id', axis = 1)

# replacing NaNs with mean values for float type
df['Linear feet of street connected to property'] = df['Linear feet of street connected to property'].fillna(\
                                                                    df['Linear feet of street connected to property'].mean())
df['Masonry veneer area in square feet'] = df['Masonry veneer area in square feet'].fillna(0.0)
df['Year garage was built'] = df['Year garage was built'].fillna(0)

# replacing NaNs with the most common value for object type
df['Electrical system'] = df['Electrical system'].fillna('SBrkr')
    
# replacing NaNs with 'NaN' value for object type
for col in ['Masonry veneer type', 'Evaluates the general condition of the basement', 'Rating of basement finished area',\
            'Rating of basement finished area (if multiple types)', 'Miscellaneous feature not covered in other categories',\
            'Type of alley access to property', 'Refers to walkout or garden level walls', 'Fireplace quality',\
            'Garage location', 'Interior finish of the garage', 'Garage quality', 'Garage condition', 'Pool quality',\
            'Fence quality', 'Evaluates the height of the basement']:
    df[col] = df[col].fillna('NA')
    
# creating dictionaries for ordinal data
bsmnt_height_dic = pd.DataFrame({'asis':['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'], 'tobe':[100.0, 90.0, 80.0, 70.0, 60.0, 0.0]})
ord_dic = pd.DataFrame({'asis':['Ex', 'Gd', 'TA', 'Fa', 'Po'], 'tobe':[1, 2, 3, 4, 5]})
subclass_dic = pd.DataFrame({'asis':[20, 30, 40, 45, 50, 60, 70, 75, 80, 85, 90, 120, 150, 160, 180, 190],\
                             'tobe':['1-STORY 1946 & NEWER ALL STYLES',\
                                    '1-STORY 1945 & OLDER',\
                                    '1-STORY W/FINISHED ATTIC ALL AGES',\
                                    '1-1/2 STORY - UNFINISHED ALL AGES',\
                                    '1-1/2 STORY FINISHED ALL AGES',\
                                    '2-STORY 1946 & NEWER',\
                                    '2-STORY 1945 & OLDER',\
                                    '2-1/2 STORY ALL AGES',\
                                    'SPLIT OR MULTI-LEVEL',\
                                    'SPLIT FOYER',\
                                    'DUPLEX - ALL STYLES AND AGES',\
                                    '1-STORY PUD (Planned Unit Development) - 1946 & NEWER',\
                                    '1-1/2 STORY PUD - ALL AGES',\
                                    '2-STORY PUD - 1946 & NEWER',\
                                    'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER',\
                                    '2 FAMILY CONVERSION - ALL STYLES AND AGES']})
    
# converting the height of the basement from categorical type to numerical
df = pd.merge(df, bsmnt_height_dic, left_on = 'Evaluates the height of the basement', right_on = 'asis', how = 'inner')
df['Evaluates the height of the basement'] = df['tobe']
df = df.drop(['asis', 'tobe'], axis = 1)

# converting the subclass of the dwelling from ordinal data to categorical
df = pd.merge(df, subclass_dic, left_on = 'Type of dwelling involved in the sale', right_on = 'asis', how = 'inner')
df['Type of dwelling involved in the sale'] = df['tobe']
df = df.drop(['asis', 'tobe'], axis = 1)

# converting categorical data to ordinal
for col in ['Evaluates the quality of the material on the exterior',\
            'Evaluates the present condition of the material on the exterior',\
            'Heating quality and condition', 'Kitchen quality']:
    df = pd.merge(df, ord_dic, left_on = col, right_on = 'asis', how = 'inner')
    df[col] = df['tobe']
    df = df.drop(['asis', 'tobe'], axis = 1)
    
# creating new target value: sale price w/o value of miscellaneous features
df['Sale price w/o misc'] = df['Sale price'] - df['$Value of miscellaneous feature']
df = df.drop(['Sale price', '$Value of miscellaneous feature'], axis = 1)

# normalizing float data
cols_to_scale = ['Linear feet of street connected to property', 'Lot size in square feet', 'Type 1 finished square feet',\
                 'Masonry veneer area in square feet', 'Evaluates the height of the basement', 'Type 2 finished square feet',
                 'Unfinished square feet of basement area', 'Total square feet of basement area', 'First Floor square feet',\
                 'Second floor square feet', 'Above grade (ground) living area square feet',\
                 'Low quality finished square feet (all floors)', 'Size of garage in square feet',\
                 'Wood deck area in square feet', 'Open porch area in square feet',\
                 'Enclosed porch area in square feet', 'Three season porch area in square feet',\
                 'Screen porch area in square feet', 'Pool area in square feet']
df_to_scale = df.loc[:, cols_to_scale]

min_max_scaler = preprocessing.MinMaxScaler()
array_scaled = min_max_scaler.fit_transform(df_to_scale.values)
df_scaled = pd.DataFrame(array_scaled)

df_scaled.columns = cols_to_scale

# replacing actual values with normalized
df = df.drop(cols_to_scale, axis = 1)
df = pd.concat([df, df_scaled], axis = 1)

# splitting dataframe into X and y
y = df['Sale price w/o misc']
X = df.drop('Sale price w/o misc', axis=1)

# feature selecting. Creating all combinations of features
X_combinations = []

cols = list(X.columns)
for i in range(0,len(cols)):
    for j in range(0, len(cols)+1):
        if j > i:
            X_combinations.append(X.iloc[:, i: j])
            
# creating list for RMSE values
rmse = []
            
# running loop over combinations of features
for count, X in enumerate(X_combinations):
    
    # saving combination of features
    cols = list(X.columns)

    # one-hot encoding categorical values
    cols_to_encode = list(X.select_dtypes('object').columns)
    df_to_encode = X.loc[:, cols_to_encode]
    df_encoded = pd.get_dummies(df_to_encode)

    # replacing actual values with encoded
    X = X.drop(cols_to_encode, axis = 1)
    X = pd.concat([X, df_encoded], axis = 1)

    # Algorithm learning

    # splitting X AND y into train and test samples
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = 7)

    # converting X and y into arrays
    X_train = X_train.values
    X_test = X_test.values
    y_train = y_train.values
    y_test = y_test.values

    # XGBoost with cross-validation
    params = {'objective':'reg:squarederror','colsample_bytree': 0.3,'learning_rate': 0.1,
                    'max_depth': 10, 'alpha': 10}

    data_dmatrix = xgb.DMatrix(data=X,label=y)
    cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                        num_boost_round=1000,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)
    
    rmse.append([(cv_results['test-rmse-mean']).tail(1), cols])
    print((cv_results['test-rmse-mean']).tail(1), count)

#     pd.set_option('display.max_columns', None)
#     df.info()

In [12]:
# for col in X.columns:
#     print(col)
pd.concat([X['Type of roof'], y['Sale price w/o misc']], axis = 1)
#     print(pd.concat(X[i], y).corr)

KeyError: 'Sale price w/o misc'

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 79 columns):
 #   Column                                                                  Non-Null Count  Dtype  
---  ------                                                                  --------------  -----  
 0   Type of dwelling involved in the sale                                   1460 non-null   object 
 1   Identifies the general zoning classification of the sale                1460 non-null   object 
 2   Type of road access to property                                         1460 non-null   object 
 3   Type of alley access to property                                        1460 non-null   object 
 4   General shape of property                                               1460 non-null   object 
 5   Flatness of the property                                                1460 non-null   object 
 6   Type of utilities available                                             1460 non

### Algorithm learning

In [18]:
# splitting dataframe into X and y
X = df.drop('Sale price w/o misc', axis = 1)
y = df['Sale price w/o misc']

# splitting X AND y into train and test samples
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = 7)

# converting X and y into arrays
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

# feature selection
for i in xrange(1,260):
    X = X[i:]
    
# XGBoost with cross-validation
    params = {'objective':'reg:squarederror','colsample_bytree': 0.3,'learning_rate': 0.1,
                    'max_depth': 10, 'alpha': 10}

    data_dmatrix = xgb.DMatrix(data=X,label=y)
    cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                        num_boost_round=10000,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

    print((cv_results["test-rmse-mean"]).tail(1))

   train-rmse-mean  train-rmse-std  test-rmse-mean  test-rmse-std
0    179153.536458     1673.524946   179414.489583    3563.979908
1    162408.098958     1576.353447   162864.177083    3572.964737
2    147396.072917     1470.858776   148178.828125    3682.550386
3    133819.833333     1392.390967   134748.575521    3738.250003
4    121579.416667     1237.517371   122958.989583    3934.504219
214    28056.061849
Name: test-rmse-mean, dtype: float64


In [12]:
pd.set_option('display.max_columns', None)
df.describe()

,Rates the overall material and finish of the house,Rates the overall condition of the house,Original construction date,Remodel date (same as construction date if no remodeling or additions),Evaluates the quality of the material on the exterior,Evaluates the present condition of the material on the exterior,Heating quality and condition,Basement full bathrooms,Basement half bathrooms,Full bathrooms above grade,Half baths above grade,Bedrooms above grade (does NOT include basement bedrooms),Kitchens above grade,Kitchen quality,Total rooms above grade (does not include bathrooms),Number of fireplaces,Year garage was built,Size of garage in car capacity,Month Sold (MM),Year Sold (YYYY),Sale price w/o misc,Linear feet of street connected to property,Lot size in square feet,Type 1 finished square feet,Masonry veneer area in square feet,Evaluates the height of the basement,Type 2 finished square feet,Unfinished square feet of basement area,Total square feet of basement area,First Floor square feet,Second floor square feet,Above grade (ground) living area square feet,Low quality finished square feet (all floors),Size of garage in square feet,Wood deck area in square feet,Open porch area in square feet,Enclosed porch area in square feet,Three season porch area in square feet,Screen porch area in square feet,Pool area in square feet,Type of dwelling involved in the sale_1-1/2 STORY - UNFINISHED ALL AGES,Type of dwelling involved in the sale_1-1/2 STORY FINISHED ALL AGES,Type of dwelling involved in the sale_1-STORY 1945 & OLDER,Type of dwelling involved in the sale_1-STORY 1946 & NEWER ALL STYLES,Type of dwelling involved in the sale_1-STORY PUD (Planned Unit Development) - 1946 & NEWER,Type of dwelling involved in the sale_1-STORY W/FINISHED ATTIC ALL AGES,Type of dwelling involved in the sale_2 FAMILY CONVERSION - ALL STYLES AND AGES,Type of dwelling involved in the sale_2-1/2 STORY ALL AGES,Type of dwelling involved in the sale_2-STORY 1945 & OLDER,Type of dwelling involved in the sale_2-STORY 1946 & NEWER,Type of dwelling involved in the sale_2-STORY PUD - 1946 & NEWER,Type of dwelling involved in the sale_DUPLEX - ALL STYLES AND AGES,Type of dwelling involved in the sale_PUD - MULTILEVEL - INCL SPLIT LEV/FOYER,Type of dwelling involved in the sale_SPLIT FOYER,Type of dwelling involved in the sale_SPLIT OR MULTI-LEVEL,Identifies the general zoning classification of the sale_C (all),Identifies the general zoning classification of the sale_FV,Identifies the general zoning classification of the sale_RH,Identifies the general zoning classification of the sale_RL,Identifies the general zoning classification of the sale_RM,Type of road access to property_Grvl,Type of road access to property_Pave,Type of alley access to property_Grvl,Type of alley access to property_NA,Type of alley access to property_Pave,General shape of property_IR1,General shape of property_IR2,General shape of property_IR3,General shape of property_Reg,Flatness of the property_Bnk,Flatness of the property_HLS,Flatness of the property_Low,Flatness of the property_Lvl,Type of utilities available_AllPub,Type of utilities available_NoSeWa,Lot configuration_Corner,Lot configuration_CulDSac,Lot configuration_FR2,Lot configuration_FR3,Lot configuration_Inside,Slope of property_Gtl,Slope of property_Mod,Slope of property_Sev,Physical locations within Ames city limits_Blmngtn,Physical locations within Ames city limits_Blueste,Physical locations within Ames city limits_BrDale,Physical locations within Ames city limits_BrkSide,Physical locations within Ames city limits_ClearCr,Physical locations within Ames city limits_CollgCr,Physical locations within Ames city limits_Crawfor,Physical locations within Ames city limits_Edwards,Physical locations within Ames city limits_Gilbert,Physical locations within Ames city limits_IDOTRR,Physical locations within Ames city limits_MeadowV,Physical locations within Ames city limits_Mitchel,Physical locations within Ames city limits_NAmes,Phy

In [16]:
df.head(n=25)

,Type of dwelling involved in the sale,Identifies the general zoning classification of the sale,Linear feet of street connected to property,Lot size in square feet,Type of road access to property,Type of alley access to property,General shape of property,Flatness of the property,Type of utilities available,Lot configuration,Slope of property,Physical locations within Ames city limits,Proximity to various conditions,Proximity to various conditions (if more than one is present),Type of dwelling,Style of dwelling,Rates the overall material and finish of the house,Rates the overall condition of the house,Original construction date,Remodel date (same as construction date if no remodeling or additions),Type of roof,Roof material,Exterior covering on house,Exterior covering on house (if more than one material),Masonry veneer type,Masonry veneer area in square feet,Evaluates the quality of the material on the exterior,Evaluates the present condition of the material on the exterior,Type of foundation,Evaluates the height of the basement,Evaluates the general condition of the basement,Refers to walkout or garden level walls,Rating of basement finished area,Type 1 finished square feet,Rating of basement finished area (if multiple types),Type 2 finished square feet,Unfinished square feet of basement area,Total square feet of basement area,Type of heating,Heating quality and condition,Central air conditioning,Electrical system,First Floor square feet,Second floor square feet,Low quality finished square feet (all floors),Above grade (ground) living area square feet,Basement full bathrooms,Basement half bathrooms,Full bathrooms above grade,Half baths above grade,Bedrooms above grade (does NOT include basement bedrooms),Kitchens above grade,Kitchen quality,Total rooms above grade (does not include bathrooms),Home functionality (Assume typical unless deductions are warranted),Number of fireplaces,Fireplace quality,Garage location,Year garage was built,Interior finish of the garage,Size of garage in car capacity,Size of garage in square feet,Garage quality,Garage condition,Paved driveway,Wood deck area in square feet,Open porch area in square feet,Enclosed porch area in square feet,Three season porch area in square feet,Screen porch area in square feet,Pool area in square feet,Pool quality,Fence quality,Miscellaneous feature not covered in other categories,$Value of miscellaneous feature,Month Sold (MM),Year Sold (YYYY),Type of sale,Condition of sale,Sale price
0,60,RL,65.000000,8450,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NA,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NA,NA,NA,0,2,2008,WD,Normal,208500
1,20,RL,80.000000,9600,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NA,NA,NA,0,5,2007,WD,Normal,181500
2,60,RL,68.000000,11250,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NA,NA,NA,0,9,2008,WD,Normal,223500
3,70,RL,60.000000,9550,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
4,60,RL,84.000000,14260,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,21